## Imports

In [1]:
import folium
from folium import plugins
import ipywidgets
import geocoder
import geopy
import numpy as np
import pandas as pd
#from vega_datasets import data as vds
#hola

## Mapa interactiu

In [18]:
# map
map_draw = folium.Map(location=[40, -99], zoom_start=4)

# draw tools
# export=True exports the drawn shapes as a geojson file
draw = plugins.Draw(export=True)

# add draw tools to map
draw.add_to(map_draw)

# display map
map_draw


## Importar dades

In [7]:
import openeo # https://openeo.org/documentation/1.0/python/#collections
import xarray as xr
import matplotlib.pyplot as plt

# Connexió al "Backend" d'on extreurèm les dades
backend = "openeo.dataspace.copernicus.eu"
conn = openeo.connect(backend).authenticate_oidc()

Authenticated using refresh token.


In [8]:
# LLista de tots els satèlits disponibles
conn.list_collections()

[{'description': 'Sentinel 3 imagery captured by OLCI sensor',
  'extent': {'spatial': {'bbox': [[-180.0, -85.0, 180.0, 85.0]]},
   'temporal': {'interval': [['2016-04-17T11:33:13Z', None]]}},
  'id': 'SENTINEL3_OLCI_L1B',
  'license': 'proprietary',
  'links': [{'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/',
    'rel': 'root',
    'type': 'application/json'},
   {'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections/sentinel-3-olci',
    'rel': 'self',
    'type': 'application/json'},
   {'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections',
    'rel': 'parent',
    'type': 'application/json'},
   {'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections/sentinel-3-olci/queryables',
    'rel': 'http://www.opengis.net/def/rel/ogc/1.0/queryables',
    'type': 'application/schema+json'},
   {'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections/sentinel-3-olci/items',
    'rel': 'items',
    'type': 'application/geo+json'}],
  'providers': [],
  'stac_extensions': ['https://stac-extensions.github.io/scientific/v1.0.0/schema.json',
   'https://stac-extensions.github.io/sat/v1.0.0/schema.json',
   'https://stac-extensions.github.io/eo/v1.0.0/schema.json'],
  'stac_version': '1.0.0',
  'title': 'Sentinel 3 OLCI'},
 {'description': 'Sentinel 3 imagery captured by SLSTR sensor',
  'extent': {'spatial': {'bbox': [[-180.0, -85.0, 180.0, 85.0]]},
   'temporal': {'interval': [['2016-04-17T11:33:13Z', None]]}},
  'id': 'SENTINEL3_SLSTR',
  'license': 'proprietary',
  'links': [{'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/',
    'rel': 'root',
    'type': 'application/json'},
   {'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections/sentinel-3-slstr',
    'rel': 'self',
    'type': 'application/json'},
   {'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections',
    'rel': 'parent',
    'type': 'application/json'},
   {'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections/sentinel-3-slstr/queryables',
    'rel': 'http://www.opengis.net/def/rel/ogc/1.0/queryables',
    'type': 'application/schema+json'},
   {'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections/sentinel-3-slstr/items',
    'rel': 'items',
    'type': 'application/geo+json'}],
  'providers': [],
  'stac_extensions': ['https://stac-extensions.github.io/scientific/v1.0.0/schema.json',
   'https://stac-extensions.github.io/sat/v1.0.0/schema.json',
   'https://stac-extensions.github.io/eo/v1.0.0/schema.json'],
  'stac_version': '1.0.0',
  'title': 'Sentinel 3 SLSTR'},
 {'description': 'Sentinel 5 Precursor imagery captured by TROPOMI sensor.\n\nThis dataset only supports loading one band at a time.',
  'extent': {'spatial': {'bbox': [[-180.0, -85.0, 180.0, 85.0]]},
   'temporal': {'interval': [['2018-04-30T00:18:50Z', None]]}},
  'id': 'SENTINEL_5P_L2',
  'license': 'proprietary',
  'links': [{'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/',
    'rel': 'root',
    'type': 'application/json'},
   {'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections/sentinel-5p-l2',
    'rel': 'self',
    'type': 'application/json'},
   {'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections',
    'rel': 'parent',
    'type': 'application/json'},
   {'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections/sentinel-5p-l2/queryables',
    'rel': 'http://www.opengis.net/def/rel/ogc/1.0/queryables',
    'type': 'application/schema+json'},
   {'href': 'https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/collections/sentinel-5p-l2/items',
    'rel': 'items',
    'type': 'application/geo+json'}],
  'providers': [],
  'stac_extensions': ['https://stac-extensions.github.io/scientific/v1.0.0/schema.json',
   'https://stac-extensions.github.io/sat/v1.0.0/schema.json',
   'https://docs.sentinel-hub.c

### ↓ Filtratge de dades ↓

In [30]:
# Determinem area de busqueda (hauriem de fer que fos automàtic amb el mapa)
spatial_extent = {
    "west": 1.936725,
    "east": 2.259583,
    "south": 41.334633,
    "north": 41.427328,
}

temporal_extent = ["2024-01-01", "2024-09-14"] # Les dates estàn en notació: aaaa-mm-dd 


# Creem una variable filtre (aka: datacube)
datacube = conn.load_collection(
    "SENTINEL2_L2A",
    spatial_extent=spatial_extent,
    temporal_extent= temporal_extent,
    bands=["B02","B03","B04"], # Els noms de les bandes cambia segons satèlit
)

# Podem fer subfiltres del filtre de la forma
avui = datacube.filter_temporal("2024-09-10", "2024-09-14")


avui = avui.mean_time()

result = avui.save_result("PNG") # Diferents formats: {'CovJSON', 'CSV', 'JSON', 'PNG', 'netCDF', 'GTiff', 'Parquet'}


# Creating a new job at the back-end by sending the datacube information.
job = result.create_job()

# Starts the job and waits until it finished to download the result.
job.start_and_wait()
job.get_results().download_files("output")

# Aquesta execució pot trigar 5 min

0:00:00 Job 'j-240914d11313493f8aeb0d5496be2750': send 'start'
0:00:13 Job 'j-240914d11313493f8aeb0d5496be2750': created (progress 0%)
0:00:19 Job 'j-240914d11313493f8aeb0d5496be2750': created (progress 0%)
0:00:25 Job 'j-240914d11313493f8aeb0d5496be2750': created (progress 0%)
0:00:33 Job 'j-240914d11313493f8aeb0d5496be2750': created (progress 0%)
0:00:43 Job 'j-240914d11313493f8aeb0d5496be2750': running (progress N/A)
0:00:55 Job 'j-240914d11313493f8aeb0d5496be2750': running (progress N/A)
0:01:11 Job 'j-240914d11313493f8aeb0d5496be2750': running (progress N/A)
0:01:30 Job 'j-240914d11313493f8aeb0d5496be2750': running (progress N/A)
0:01:54 Job 'j-240914d11313493f8aeb0d5496be2750': running (progress N/A)
0:02:24 Job 'j-240914d11313493f8aeb0d5496be2750': running (progress N/A)
0:03:01 Job 'j-240914d11313493f8aeb0d5496be2750': running (progress N/A)
0:03:48 Job 'j-240914d11313493f8aeb0d5496be2750': running (progress N/A)
0:04:47 Job 'j-240914d11313493f8aeb0d5496be2750': finished (progr

[PosixPath('output/out.png'), PosixPath('output/job-results.json')]